# Self Health Study

This study will help me learn how food and excercise are affecting my weight and overall well-being. I am using historical data (unfortenately not continuous) exported from myfitnesspal. It covers a period of 6 years of "struggling".

First I will read the data I have into a pandas dataframe

In [97]:
import pandas as pd

excercise = pd.read_csv('myfitnesspal/Exercise-Summary.csv')
measurment = pd.read_csv('myfitnesspal/Measurement-Summary.csv')
nutrition = pd.read_csv('myfitnesspal/Nutrition-Summary.csv')

Showing the structure of the excersice dataframe

In [98]:
excercise

,Date,Exercise,Type,Exercise Calories,Exercise Minutes,Sets,Reps Per Set,Kilograms,Steps
0,2012-11-10,Dips,Strength,NaN,NaN,7.0,10.0,36.3,NaN
1,2012-11-10,"Rowing, stationary, very vigorous effort",Cardio,227.0,22.0,NaN,NaN,NaN,NaN
2,2012-11-10,"Step-ups, vigorous",Cardio,251.0,20.0,NaN,NaN,NaN,NaN
3,2012-11-11,Chin-Ups,Strength,NaN,NaN,3.0,8.0,36.3,NaN
4,2012-11-11,Dips,Strength,NaN,NaN,5.0,10.0,36.3,NaN
5,2012-11-11,"Rowing, stationary, very vigorous effort",Cardio,300.0,30.0,NaN,NaN,NaN,NaN
6,2012-11-11,"Stationary bike, light effort (bicycling, cycl...",Cardio,52.0,5.0,NaN,NaN,NaN,NaN
7,2012-11-11,"Step-ups, vigorous",Cardio,225.0,20.0,NaN,NaN,NaN,NaN
8,2012-11-12,"Rowing, stationary, very vigorous effort",Cardio,200.0,30.0,NaN,NaN,NaN,NaN
9,2012-11-12,"Stationary bike, light effort (bicycling, cycl...",Cardio,60.0,5.0,NaN,NaN,NaN,NaN


Showing the structure of the measurment dataframe

In [99]:
measurment

,Date,Hips,Neck,Waist,Weight
0,2012-11-10,NaN,NaN,NaN,113.8
1,2012-11-11,NaN,NaN,NaN,114.1
2,2012-11-13,NaN,NaN,NaN,113.5
3,2012-11-18,NaN,NaN,NaN,113.9
4,2012-11-24,NaN,NaN,NaN,114.2
5,2012-11-25,NaN,NaN,NaN,114.2
6,2012-12-03,NaN,NaN,NaN,115.0
7,2012-12-05,NaN,NaN,NaN,113.8
8,2012-12-19,NaN,NaN,NaN,114.1
9,2012-12-29,NaN,NaN,NaN,115.2


Showing the structure of the nutrition dataframe

In [100]:
nutrition

,Date,Meal,Calories,Fat (g),Saturated Fat,Polyunsaturated Fat,Monounsaturated Fat,Trans Fat,Cholesterol,Sodium (mg),Potassium,Carbohydrates (g),Fiber,Sugar,Protein (g),Vitamin A,Vitamin C,Calcium,Iron
0,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,0.0,60.7,0.0,53.9,18.5,0.0,0.0,0.0,0.0
1,2012-11-11,Lunch,805.0,35.0,14.7,0.0,11.9,0.0,182.0,2597.0,0.0,91.0,22.4,0.0,35.0,0.0,0.0,0.0,0.0
2,2012-11-11,Dinner,614.8,41.6,11.4,3.6,19.4,0.0,44.6,656.8,1638.6,50.4,8.4,30.5,18.8,78.4,114.7,44.2,14.1
3,2012-11-11,Snacks,72.0,0.0,0.0,0.0,0.0,0.0,0.0,133.0,0.0,17.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0
4,2012-11-12,Lunch,840.0,59.0,13.0,0.0,0.0,0.0,153.0,1787.0,672.0,81.5,2.0,0.0,56.3,0.0,0.0,2.0,10.0
5,2012-11-12,Dinner,585.9,38.8,9.4,3.7,19.4,0.0,44.6,600.8,2081.0,54.9,9.9,29.1,16.0,84.7,128.8,49.0,18.8
6,2012-11-12,Snacks,1508.0,91.1,18.0,0.0,0.0,0.0,0.0,122.2,0.0,106.8,9.8,83.8,19.4,0.0,0.0,4.0,0.0
7,2012-11-13,Breakfast,430.0,10.8,1.6,1.0,2.0,0.0,0.0,167.0,210.0,66.6,8.6,30.6,18.0,15.0,0.0,0.0,50.0
8,2012-11-13,Lunch,1610.0,70.0,29.4,0.0,23.8,0.0,364.0,5194.0,0.0,182.0,44.8,0.0,70.0,0.0,0.0,0.0,0.0
9,2012-11-13,Dinner,1355.6,97.6,25.0,9.8,47.0,0.0,937.2,1427.6,3055.2,82.2,14.4,45.0,51.6,147.0,187.0,96.4,38.4


Now I will merge all the data into a single dataframe. I will use the pandas **merge** method so I can properly arrange my data according the Date in which it was entered.

In [101]:
df1 = pd.merge(nutrition, excercise, on='Date')
df2 = pd.merge(df1, measurment, on='Date')

df2

,Date,Meal,Calories,Fat (g),Saturated Fat,Polyunsaturated Fat,Monounsaturated Fat,Trans Fat,Cholesterol,Sodium (mg),...,Exercise Calories,Exercise Minutes,Sets,Reps Per Set,Kilograms,Steps,Hips,Neck,Waist,Weight
0,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,NaN,NaN,3.0,8.0,36.3,NaN,NaN,NaN,NaN,114.1
1,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,NaN,NaN,5.0,10.0,36.3,NaN,NaN,NaN,NaN,114.1
2,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,300.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.1
3,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,52.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.1
4,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,225.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.1
5,2012-11-11,Lunch,805.0,35.0,14.7,0.0,11.9,0.0,182.0,2597.0,...,NaN,NaN,3.0,8.0,36.3,NaN,NaN,NaN,NaN,114.1
6,2012-11-11,Lunch,805.0,35.0,14.7,0.0,11.9,0.0,182.0,2597.0,...,NaN,NaN,5.0,10.0,36.3,NaN,NaN,NaN,NaN,114.1
7,2012-11-11,Lunch,805.0,35.0,14.7,0.0,11.9,0.0,182.0,2597.0,...,300.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.1
8,2012-11-11,Lunch,805.0,35.0,14.7,0.0,11.9,0.0,182.0,2597.0,...,52.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.1
9,2012-11-11,Lunch,805.0,35.0,14.7,0.0,11.9,0.0,182.0,2597.0,...,225.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.1


I know that my measurments for hips, neck and waist are total garbage as I measured them only once and it wasn't even a proper measurment, thus I will simply remove them from the dataframe.

In [102]:
df2.drop(['Hips','Neck','Waist'], axis=1, inplace=True)
df2.head()

,Date,Meal,Calories,Fat (g),Saturated Fat,Polyunsaturated Fat,Monounsaturated Fat,Trans Fat,Cholesterol,Sodium (mg),...,Iron,Exercise,Type,Exercise Calories,Exercise Minutes,Sets,Reps Per Set,Kilograms,Steps,Weight
0,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,0.0,Chin-Ups,Strength,NaN,NaN,3.0,8.0,36.3,NaN,114.1
1,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,0.0,Dips,Strength,NaN,NaN,5.0,10.0,36.3,NaN,114.1
2,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,0.0,"Rowing, stationary, very vigorous effort",Cardio,300.0,30.0,NaN,NaN,NaN,NaN,114.1
3,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,0.0,"Stationary bike, light effort (bicycling, cycl...",Cardio,52.0,5.0,NaN,NaN,NaN,NaN,114.1
4,2012-11-11,Breakfast,574.0,28.4,18.7,0.0,0.0,0.0,0.0,186.0,...,0.0,"Step-ups, vigorous",Cardio,225.0,20.0,NaN,NaN,NaN,NaN,114.1


Ok, next thing I want to do is to add a column which shows my weight change compared to the previous entry (day). I will name it 'dW'.

In [103]:
df2['dW'] = df2['Weight'].diff(periods=1)

I know that my data is not continuous and if I use it this way I will jepardize the whole purpose of the study. However I know that I have been consistent in my entries (weight, nutrition and excercise) since **4th December 2017**. Knowing this I will simply make a new working dataframe with the entries made after that.

In [111]:
import datetime as dt

df2['Date'] = pd.to_datetime(df2['Date'])

df = df2[df2['Date'] >= dt.datetime(2017,12,4)]

df

,Date,Meal,Calories,Fat (g),Saturated Fat,Polyunsaturated Fat,Monounsaturated Fat,Trans Fat,Cholesterol,Sodium (mg),...,Exercise,Type,Exercise Calories,Exercise Minutes,Sets,Reps Per Set,Kilograms,Steps,Weight,dW
139,2017-12-08,Dinner,935.0,23.5,1.9,0.0,0.0,0.0,0.0,2500.0,...,MFP Android Google calorie adjustment,Cardio,563.0,1.0,NaN,NaN,NaN,9634.0,107.5,0.5
140,2017-12-08,Dinner,935.0,23.5,1.9,0.0,0.0,0.0,0.0,2500.0,...,"Running (jogging), 5 mph (12 min mile)",Cardio,230.0,16.0,NaN,NaN,NaN,NaN,107.5,0.0
141,2017-12-08,Snacks,206.0,0.4,0.1,0.1,0.0,0.0,0.0,4.2,...,MFP Android Google calorie adjustment,Cardio,563.0,1.0,NaN,NaN,NaN,9634.0,107.5,0.0
142,2017-12-08,Snacks,206.0,0.4,0.1,0.1,0.0,0.0,0.0,4.2,...,"Running (jogging), 5 mph (12 min mile)",Cardio,230.0,16.0,NaN,NaN,NaN,NaN,107.5,0.0
143,2017-12-09,Dinner,1204.7,55.3,19.7,2.2,12.8,0.0,0.0,3405.3,...,MFP Android Google calorie adjustment,Cardio,612.0,1.0,NaN,NaN,NaN,6631.0,106.8,-0.7
144,2017-12-09,Snacks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,MFP Android Google calorie adjustment,Cardio,612.0,1.0,NaN,NaN,NaN,6631.0,106.8,0.0
145,2017-12-10,Dinner,1365.7,81.6,29.4,2.2,12.8,0.0,0.0,2047.0,...,MFP Android Google calorie adjustment,Cardio,532.0,1.0,NaN,NaN,NaN,9932.0,106.7,-0.1
146,2017-12-11,Dinner,861.1,53.2,7.6,2.3,12.8,0.0,117.8,157.8,...,MFP Android Google calorie adjustment,Cardio,954.0,1.0,NaN,NaN,NaN,17256.0,106.2,-0.5
147,2017-12-12,Dinner,676.1,41.6,0.0,0.1,0.0,0.0,118.5,94.3,...,MFP Android Google calorie adjustment,Cardio,1162.0,1.0,NaN,NaN,NaN,22392.0,105.5,-0.7
148,2017-12-13,Dinner,1467.8,72.1,6.3,4.0,22.4,0.0,132.3,989.0,...,MFP Android Google calorie adjustment,Cardio,892.0,1.0,NaN,NaN,NaN,16341.0,105.0,-0.5


So now I already have the database in hand and I can start exploring it.